## Generación de par de claves y transacciones
### Alejandro Manuel López Gómez


Este código simula la aplicación de usuario de un endpoint de la red blockchain y demuestra las funcionalidades básicas que debería tener (Generacion de par de claves y firma de transacciones). Este proceso debería ser transparente para el usuario, donde solamente se le pediría su nombre y contenido de la transacción.

- Al ejecutar el código por primera vez, debe indicarse que no se tiene un par de claves generado, ya que el programa generará un par nuevo. En ejecuciones posteriores se debe pasar el par de claves generado anteriormente.
- Aunque en este código no se realiza la asociación usuario - módulo, esta si será realizada al registrar el usuario con el módulo generado. Por tanto es importante que el usuario recuerde su usuario y módulo a la hora de crear transacciones.

Para realizar la firma, se debe obtener el hash de las transacciones y encriptarlo con la clave privada. La clave pública es la combinación de e y n. Como el exponente e ya es conocido por los nodos (normalmente se emplea un valor estándar como 65537), solamente es necesario al registrarse enviar el valor del módulo n. El valor de la clave privada d nunca debe ser expuesto, ya que es el que se emplea para realizar las firmas.

In [1]:
from Crypto.Util.number import getPrime
import libnum
import json
import hashlib

# Exponent
e = 0x10001

generated = input("Do you have a generated RSA key pair? (Y/N) ")

try:
    if generated.upper() == "Y":
        n = int(input("Modulus value> "))
        d = int(input("Private key> "))
    else:
        print("Generating key pair...")
        # RSA prime number generation
        p, q = [getPrime(512) for i in range(2)]

        # Modulus generation (with the exponent e it is know as the public key)
        n = p * q

        # private key generation
        phi = (p - 1) * (q - 1)
        d = libnum.invmod(e,phi)

        print("\nGENERATED KEY PAIR INFORMATION (REMEMBER TO SAVE THIS!!)")
        print("\n=========== PUBLIC KEY INFORMATION ===========")
        print("\nModulus (PASTE this number when REGISTERING USER):",n)
        print("\n=========== PRIVATE KEY INFORMATION ===========")
        print("\nDo NOT share this value:",d)

    print("\n=========== TRANSACTION GENERATION ===========")
    finish = False
    i = 0
    author = input("\nIntroduce your username (REMEMBER THIS FOR FUTURE TRANSACTIONS)>")

    while finish == False:
        i += 1
        print("\n\nCreating Transaction ", i, " for user ", author)
        transaction = input("\nIntroduce your transaction content> ")

        # To sign a message, obtain the hash and encrypt with private key
        transaction = {"author" : author, "content" : transaction}

        # Hash of transaction
        transaction_hash = int(hashlib.sha256(json.dumps(transaction,sort_keys=True).encode('utf8')).hexdigest(),16)
        print("\nTransaction hash (To be encrypted):",transaction_hash)

        # Creating signature by encrypting the hash
        signature = pow(transaction_hash,d,n)

        print("\nSignature (PASTE this number as your TRANSACTION SIGNATURE):",signature)

        print_poc = input("Print decrypted signature? (Y/N) (Proof of Concept) ")
        if (print_poc.upper() == 'Y'):
            # Proof of concept: by employing the public key to decrypt we obtain the same hash
            decrypted = pow(signature,e,n)
            print("\nDecrypted Signature:",decrypted)
            # This decrypted hash would now be compared with the transaction hash to verify the signature

        is_finished = input("\nDo you want to create another transaction signature? (Y/N) ")
        if (is_finished.upper() != 'Y'):
            print("\nGoodbye (REMEMBER TO SAVE YOUR GENERATED KEY PAIR!)")
            finish = True
            
except ValueError:
    print("An unexpected character was given as an input")
    raise

Do you have a generated RSA key pair? (Y/N) N
Generating key pair...

GENERATED KEY PAIR INFORMATION (REMEMBER TO SAVE THIS!!)

=========== PUBLIC KEY INFORMATION ===========

Modulus (PASTE this number when REGISTERING USER): 106360015573337743395292516124736373872553915967421616071445511579665909273661469268346392112178028629779670467239007124455885928592039064886102839980550587825099043650203963772592668407178112493541286250854473469520833719987521038983914554750834263939414979678789031805715597312068807325968822016604580523701

=========== PRIVATE KEY INFORMATION ===========

Do NOT share this value: 32029559902885296910897555552402415044154051688863283561744489624735437774463017188459715805208440621898035862816867488720285711684887666273893001660987625082408328884255279110426055399427706141413355458970839439753930082285078961894532780893795495742597398198041036644246381464598380743277006485563748156673

=========== TRANSACTION GENERATION ===========

Introduce your username (REMEM